In [ ]:
import cv2
import rospy
import message_filters
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image
import numpy as np

In [ ]:
class Image_Converter(object):
    
    def __init__(self, color_topic, depth_topic=None):
        self.bridge = CvBridge()
        
        if depth_topic:
            image_sub = message_filters.Subscriber(color_topic, Image)
            depth_sub = message_filters.Subscriber(depth_topic, Image)
            
            self.timeSync = message_filters.ApproximateTimeSynchronizer(
                            [image_sub, depth_sub], 1, 0.5)
            self.timeSync.registerCallback(self.callback)
            
        #self.windows = cv2.namedWindow('Image', 0)
            
    def callback(self, image_msg, depth_msg):
        header = image_msg.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs
        print('Header: %s' % header)
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, "bgr8")
            depth_data = self.bridge.imgmsg_to_cv2(depth_msg, "passthrough")
        except CvBridgeError as e:
            print(e)     

In [ ]:
rospy.init_node('Image_Viewer', anonymous=True)
vis = Image_Converter('/rs01/color/image_rect_color_resize',
                      '/rs01/aligned_depth_to_color/image_raw_resize')

try:
    rospy.spin()
except KeyboardInterrupt:
    print("Image Converter Shutting Down.")